In [3]:
import os
import xml.etree.ElementTree as ET
import numpy as np
import cv2

def convert_xml_to_mask(path_to_xml_data, path_to_bi_maps):
    # Ensure the output directory exists
    if not os.path.exists(path_to_bi_maps):
        os.makedirs(path_to_bi_maps)

    # Iterate over all XML files in the input directory
    for xml_file in os.listdir(path_to_xml_data):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(path_to_xml_data, xml_file)
            print(f"Processing: {xml_path}")

            # Parse the XML file
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
            except ET.ParseError as e:
                print(f"Error parsing {xml_file}: {e}")
                continue

            # Get the image dimensions
            size = root.find('size')
            if size is None:
                print(f"Skipping {xml_file}: No <size> tag found.")
                continue

            try:
                width = int(size.find('width').text.strip())
                height = int(size.find('height').text.strip())
            except (AttributeError, ValueError):
                print(f"Skipping {xml_file}: Invalid width/height.")
                continue

            # Create an empty mask image
            mask = np.zeros((height, width), dtype=np.uint8)

            # Iterate over all objects in the XML file
            for obj in root.findall('object'):
                polygon = obj.find('polygon')
                if polygon is None:
                    print(f"Skipping {xml_file}: No <polygon> found in object.")
                    continue

                points = []
                i = 1  # Start with x1, y1
                while True:
                    x_tag = f"x{i}"
                    y_tag = f"y{i}"

                    x_elem = polygon.find(x_tag)
                    y_elem = polygon.find(y_tag)

                    if x_elem is None or y_elem is None:
                        break  # Stop when no more points exist

                    try:
                        x = int(float(x_elem.text.strip()))  # Convert float to int
                        y = int(float(y_elem.text.strip()))
                        points.append((x, y))
                    except (AttributeError, ValueError):
                        print(f"Skipping point in {xml_file}: Invalid x/y value.")
                        continue
                    
                    i += 1  # Move to next numbered point

                if len(points) < 3:
                    print(f"Skipping {xml_file}: Not enough points for a polygon.")
                    continue

                # Convert to NumPy array with proper shape
                points = np.array(points, dtype=np.int32)

                # Ensure correct shape for cv2.fillPoly
                points = points.reshape((-1, 1, 2))

                # Draw the polygon on the mask image
                cv2.fillPoly(mask, [points], 255)

            # Save the mask image
            mask_filename = os.path.splitext(xml_file)[0] + '_mask.jpg'
            mask_filepath = os.path.join(path_to_bi_maps, mask_filename)
            cv2.imwrite(mask_filepath, mask)
            print(f"Saved: {mask_filepath}")

# Example usage
convert_xml_to_mask('Zebra_fish_data/test_data_withxml/test', 'Zebra_fish_data/test_data_withxml/masks')


Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-01.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-01_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-03.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-03_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-04.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-04_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-05.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-05_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-06.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-06_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/test\PlatteA_119hpf_pr_20-07.xml
Saved: Zebra_fish_data/test_data_withxml/masks\PlatteA_119hpf_pr_20-07_mask.jpg
Processing: Zebra_fish_data/test_data_withxml/